In [5]:
from pyflink.table import expressions as fl
from pyflink.table import EnvironmentSettings, TableEnvironment

env = EnvironmentSettings.in_streaming_mode()
flink = TableEnvironment.create(env)
flink

In [4]:
flink.execute_sql("""
    create table datagen (
        id int,
        data string
    ) with (
        'connector' = 'datagen',
        'fields.id.kind' = 'sequence',
        'fields.id.start' = '1',
        'fields.id.end' = '10')
""")

In [6]:
flink.execute_sql("""
    create table print (
        id int,
        data string
    ) with (
        'connector' = 'print'
    )
""")

In [7]:
source_table = flink.sql_query("select * from datagen")

source_table.execute().print()

+----+-------------+--------------------------------+
| op |          id |                           data |
+----+-------------+--------------------------------+
| +I |           1 | 416333293291edd52f46b454608... |
| +I |           2 | 0967f7224dc538165bae9cf42ef... |
| +I |           3 | 6172f3199414866f48991d1222c... |
| +I |           4 | 2d7337f3f1c69c0f59d47b81b55... |
| +I |           5 | e4f633db590479840b82bc1328d... |
| +I |           6 | 6373332228809b3bcce0362efcf... |
| +I |           7 | 26f35115e0cf12206be32be46de... |
| +I |           8 | c0e173feb182d0fb7ac0b1c7cd7... |
| +I |           9 | 46e371c7f24cea9715d6c9eeedc... |
| +I |          10 | 456031289a536b36f092771dfa2... |
+----+-------------+--------------------------------+
10 rows in set


In [8]:
result_table = source_table.select(fl.col("id") + 1, fl.col("data"))

result_table.execute().print()

+----+-------------+--------------------------------+
| op |         _c0 |                           data |
+----+-------------+--------------------------------+
| +I |           2 | aef415d15f01caf46d9f74b5c12... |
| +I |           3 | 0ce555dba14f0213e8e2ab6c6ca... |
| +I |           4 | cee5b57323a3025e5af5f20a5de... |
| +I |           5 | 26f311eac0a33f4a0e37930a54a... |
| +I |           6 | 21703398f09e44ff19d7cb75c0b... |
| +I |           7 | faf1d538a77360b8f96b712e397... |
| +I |           8 | ab7155541e4d57d4850bc8014ef... |
| +I |           9 | e7721178c66636d3c5b8df60b8b... |
| +I |          10 | 08e0d5dcdaf3afa5ae1611697ad... |
| +I |          11 | b4f49348f6832e09be4ad30aa4d... |
+----+-------------+--------------------------------+
10 rows in set


In [9]:
result_table.execute_insert("print").wait()

1> +I[2, 6172d3f1baafe1620d47490ae721af0bc300e89cbcec2012edb119a2608e396d12c1ce646b2586ca08d8ddaee588fe9ab47b]
2> +I[3, 2771714e5d7ab16e2a1e825ed0e79da81ce8a2235033b1ca87796081b87406d8e662f9663c102f90bde3372d9edfae9b4b1e]
7> +I[8, 72f4f196595a4a8e66142648677d7bde1d0275c0e52923094496d7a427d0d986844b52f42e10af8a9451f3d0175c2565f098]
5> +I[6, 7ef6554bf6c0e5e97a4b5798a5c74eef7591c4c33751b1a933a2847a0d54db96bd8e7c42c76c5a37e8960c25063f75f074ba]
4> +I[5, a772f1ccd84dd9bab0146b63db0c6bd72aeeb6108e813bae6255475eb319b259b473e14449da257adb19e1a66ed8740fafec]
6> +I[7, db67d7a5e71b5ab38fff21224203459f474ccd3b41d7f0f8f0cf684123e391d6ca4230ae4eedff625712a9b504bb89d0eabc]
8> +I[9, e97757cacbc32bc5ffb431c45123b386fcf8ebc67a315e0e6d3840c34626ecb52d1cfa263a0c1dece1eb69ab2e5225d42f6f]
10> +I[11, 7b7095a60ed2428633399491cc8e764cfac470f10bbd7fdfdbc85a81610646360d407aa5e502614400c27610cb037dcd2c20]
3> +I[4, 70f4b1e7634d97c3fde5ccf3d446618ce175a69b40a733ce955d4a6310af61a810210c2c75f903a04d62c5ccbb7f708c6945]

In [12]:
flink.execute_sql("insert into print select * from datagen").wait()

2> +I[2, 9561d1ed97342bd9191b1fe7d31d1830496f543ca622d39cef872518d42d4c19ef3c4fd2b496a729bc765880d49c6279779d]
1> +I[1, f177b10547fa0a128c3f420166f0c61344818930ce1a3dd63f56558791647babd1a05b1466c4f196bad21fee86a28d2f84a9]
4> +I[4, 8ee715adaea0e57b496f3503a999d2dec01f38dacb2c57b0e4da5d2771a0025f60f352de27cbcc460a4523054855e160c431]
5> +I[5, 2773b183b6a1bcd59cdf55521be3814a81fb2dad78d3b19b3b3ea6b14e6f2d3cbc8fc829d2927bba2f3ef21e90019a945704]
6> +I[6, 68f6174553e37d0461801603aff4eae83606e2b9a811d764d18ca5371105c9dac8259c746c7b047e58d2437f5db80fa87609]
3> +I[3, 4d67f5ce11f0c2ba4c9396e138c2ba8a322b2d8ba1049d0662094df7762c81e7d36eca07a42c053d4fc3cec62f78325670e8]
7> +I[7, 5764132341217adcea201602d163c821196daadded6be4ca7890d527b0a2c52fd86433dba99c4c456a4e8e2994551086068a]
8> +I[8, d76473d07e1fe025ebd9e807b03077fc8a2cccbc0b2b9762a3dfb22398e11d7fe3a6d3efd15c081f9cab0e7051c1e04bdb85]
10> +I[10, eaf0b5ec13cd93d2dd17acb272a72e7632b925ff9272736a507883c2ce3d50eb069d4b173d83ae289470d7495ccdd9a00d9d]

In [14]:
flink.execute_sql("select * from datagen").print()

+----+-------------+--------------------------------+
| op |          id |                           data |
+----+-------------+--------------------------------+
| +I |           1 | 2b72350aadcbbc581d6801ded99... |
| +I |           2 | 5566f361d6e92d11ef8e057255b... |
| +I |           3 | 5960b56006bf32ea3b2888c07fb... |
| +I |           4 | 0566b354e90eae37c7e4af9d4da... |
| +I |           5 | 14e4f34fe8f8371d5fe40534b31... |
| +I |           6 | 20f033e990a2598ebf0133c2f93... |
| +I |           7 | 10e3b3674831c504abb1765c779... |
| +I |           8 | 1d64f55f9ad6b4fe25149db6cb2... |
| +I |           9 | 1362132542c22d008bb4d0bf14e... |
| +I |          10 | 60f4f17bb9ae46596b562b0919e... |
+----+-------------+--------------------------------+
10 rows in set
